# Decorators

Today, we'll briefly cover [decorators](https://www.geeksforgeeks.org/decorators-in-python/), which are functions that wrap functions, and come with [syntactic sugar](https://en.wikipedia.org/wiki/Syntactic_sugar) which lets you write things like

```python
@time_wrapper
def my_function(x):
    ...
    
y = my_function(x)
```
```
> 1.1 seconds elapsed.
```

Again, we want to write a function that wraps another function.  We can define this as

In [2]:
from time import time

def time_wrapper(f):
    
    def inner_wrapper(*args, **kwargs):
        t0 = time()
        ret = f(*args, **kwargs)
        t1 = time()
        print("{} sec. elapsed".format(t1 - t0))
        return ret
    
    return inner_wrapper

Here's how we might use our wrapper:

In [3]:
def generate_random(n):
    """
    return a random numpy vector
    """
    return np.random.rand(n)

generate_random = time_wrapper(generate_random)

generate_random(100000)

0.0021889209747314453 sec. elapsed


array([0.24671109, 0.39224309, 0.82490506, ..., 0.34970352, 0.12396696,
       0.40322369])

one problem with this is that we had to go through the trouble of wrapping the function. Instead, we can just write

In [4]:
@time_wrapper
def generate_random2(n):
    """
    return a random numpy vector
    """
    return np.random.rand(n)

generate_random2(100000)

0.0018007755279541016 sec. elapsed


array([0.87521796, 0.47016823, 0.62784667, ..., 0.13415162, 0.56107656,
       0.38131295])

This is interpreted like what we did above, but is a bit easier to read.

Another problem we may encounter is that we now can't access the docstring we wrote

In [5]:
help(generate_random2)

Help on function inner_wrapper in module __main__:

inner_wrapper(*args, **kwargs)



One way to solve this is to use the `wraps` decorator from the [functools](https://docs.python.org/3/library/functools.html) package.

In [6]:
from functools import wraps

def time_wrapper2(f):
    
    @wraps(f)
    def inner_wrapper(*args, **kwargs):
        t0 = time()
        ret = f(*args, **kwargs)
        t1 = time()
        print("{} sec. elapsed".format(t1 - t0))
        return ret
    
    return inner_wrapper

@time_wrapper2
def generate_random3(n):
    """
    return a random numpy vector
    """
    return np.random.rand(n)

generate_random3(100000)

0.0010859966278076172 sec. elapsed


array([0.99595756, 0.02890335, 0.88528414, ..., 0.36966219, 0.17925142,
       0.77181316])

now, we can access the metadata from `generate_random3`, such as the docstring

In [7]:
help(generate_random3)

Help on function generate_random3 in module __main__:

generate_random3(n)
    return a random numpy vector



As you might guess, the `@wraps` decorator copies the class metadata from `f` to the `inner_wrapper` function.